In [1]:
import pandas as pd
import re
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import pickle
from datetime import datetime

### Reading KDD train file
Drop rows with null in any of the important columns

In [2]:
folder = 'data/kdd/2005-2006'

In [3]:
%%time
df_all = pd.read_csv('%s/algebra_2005_2006_train.txt' % folder, delimiter='\t')
print(df_all.shape)

(809694, 19)
CPU times: user 7.59 s, sys: 1.75 s, total: 9.34 s
Wall time: 4.69 s


In [4]:
df_all

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,Step Duration (sec),Correct Step Duration (sec),Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects,KC(Default),Opportunity(Default)
0,1,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG4-FIXED,1,3(x+2) = 15,2005-09-09 12:24:35.0,2005-09-09 12:24:49.0,2005-09-09 12:25:15.0,2005-09-09 12:25:15.0,40.0,NaN,40.0,0,2,3,1,[SkillRule: Eliminate Parens; {CLT nested; CLT...,1
1,2,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG4-FIXED,1,x+2 = 5,2005-09-09 12:25:15.0,2005-09-09 12:25:31.0,2005-09-09 12:25:31.0,2005-09-09 12:25:31.0,16.0,16.0,NaN,1,0,0,1,"[SkillRule: Remove constant; {ax+b=c, positive...",1~~1
2,3,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG40,1,2-8y = -4,2005-09-09 12:25:36.0,2005-09-09 12:25:43.0,2005-09-09 12:26:12.0,2005-09-09 12:26:12.0,36.0,NaN,36.0,0,2,3,1,"[SkillRule: Remove constant; {ax+b=c, positive...",2
3,4,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG40,1,-8y = -6,2005-09-09 12:26:12.0,2005-09-09 12:26:34.0,2005-09-09 12:26:34.0,2005-09-09 12:26:34.0,22.0,22.0,NaN,1,0,0,1,"[SkillRule: Remove coefficient; {ax+b=c, divid...",1~~1
4,5,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG40,2,-7y-5 = -4,2005-09-09 12:26:38.0,2005-09-09 12:28:36.0,2005-09-09 12:28:36.0,2005-09-09 12:28:36.0,118.0,118.0,NaN,1,0,0,1,"[SkillRule: Remove constant; {ax+b=c, positive...",3~~1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809689,1080612,IQQo3367x0,"Unit ES_02, Section ES_02-5",EG40,4,-7+2x+7 = 4+7,2006-03-09 10:58:05.0,2006-03-09 10:58:13.0,2006-03-09 10:58:13.0,2006-03-09 10:58:13.0,8.0,8.0,NaN,1,0,0,1,[SkillRule: Consolidate vars with coeff; CLT],5
809690,1080613,IQQo3367x0,"Unit ES_02, Section ES_02-5",EG40,4,2x = 4+7,2006-03-09 10:58:13.0,2006-03-09 10:58:18.0,2006-03-09 10:58:18.0,2006-03-09 10:58:18.0,5.0,5.0,NaN,1,0,0,1,[SkillRule: Consolidate vars with coeff; CLT],6
809691,1080614,IQQo3367x0,"Unit ES_02, Section ES_02-5",EG40,4,2x = 11,2006-03-09 10:58:18.0,2006-03-09 10:58:22.0,2006-03-09 10:58:39.0,2006-03-09 10:58:39.0,21.0,NaN,21.0,0,1,3,1,[SkillRule: Remove positive coefficient; {ax/b...,1~~4
809692,1080615,IQQo3367x0,"Unit ES_02, Section ES_02-5",EG40,4,2x/2 = 11/2,2006-03-09 10:58:39.0,2006-03-09 10:58:44.0,2006-03-09 10:58:44.0,2006-03-09 10:58:44.0,5.0,5.0,NaN,1,0,0,1,NaN,NaN


In [5]:
df = df_all[['Anon Student Id', 'Correct First Attempt', 'Problem Hierarchy', 'Problem Name', 'Problem View', 
             'Step Name', 'KC(Default)', 'First Transaction Time',
             # 'Correct Transaction Time'
            ]]

In [6]:
df = df.dropna()

In [7]:
df.shape

(607025, 8)

### Preprocessing
Transform dataframe in a format understood by the BKT module

In [8]:
skipped = []
def get_kcs(row):
    text = row['KC(Default)'].replace(',','')
    kc_list = text.split('~~')
    kc = []
    for item in kc_list:
        # Option 1: with SkillRule prefix
        found_opt1 = re.findall(r'^\[SkillRule: ([\w ]+[ ,/?-]*[\w ]*);.*', item, flags=re.M)
        # Option 2: regular words (hyphenized or spaced)
        found_opt2 = re.findall(r'^([\w -]+).*', item, flags=re.M)
        # Option 3: get what couldn't be extracted from previous patterns
        found_opt3 = re.findall(r'^\[SkillRule: (.*);.*', item, flags=re.M)
        if found_opt1:
            kc.append(found_opt1[0])
        elif found_opt2:
            kc.append(found_opt2[0])
        elif found_opt3:
            kc.append(found_opt3[0])
        else:
            skipped.append(item)
    return kc

def get_steps(row):
    step = row['Problem Hierarchy'] + '_' + row["Problem Name"] + '_' + row['Step Name']
    return step

Separate KCs in a list and create an identifier name for the step

In [9]:
%%time
df['KC'] = df.apply(get_kcs, axis=1)
df['step'] = df.apply(get_steps, axis=1)

CPU times: user 32.6 s, sys: 1.02 s, total: 33.6 s
Wall time: 33.6 s


In [10]:
%%time
kcs = []
for row in df['KC'].iteritems():
    kcs.extend(row[1])
kcs = list(set(kcs))
steps = df['step'].unique().tolist()

def create_data(row):
    question_id = steps.index(row['step'])
    return question_id

df['question_id'] = df.apply(create_data, axis=1)

CPU times: user 5min 11s, sys: 250 ms, total: 5min 11s
Wall time: 5min 11s


Save KCs

In [11]:
with open('%s/kdd_2005_2006_kcs_time.pkl' % folder, 'wb') as pklfile:
    pickle.dump(kcs, pklfile)
len(kcs)

110

Construct Q-Matrix

In [12]:
%%time
q_matrix = np.zeros((len(steps), len(kcs)))

for idx, item in df.iterrows():
    for kc in item['KC']:
        kc_col = kcs.index(kc)
        q_matrix[item['question_id'], kc_col] = 1
    
print(q_matrix.shape)

(176630, 110)
CPU times: user 1min 15s, sys: 656 ms, total: 1min 16s
Wall time: 1min 12s


Save data

In [13]:
%%time
with open('%s/kdd_2005_2006_df_time.pkl' % folder, 'wb') as pklfile:
    pickle.dump(df, pklfile)
with open('%s/kdd_2005_2006_q_matrix_time.pkl' % folder, 'wb') as pklfile:
    pickle.dump(q_matrix, pklfile)

CPU times: user 1 s, sys: 859 ms, total: 1.86 s
Wall time: 2.32 s


In [14]:
%%time
# Data matrix
data = df[['Correct First Attempt', 'Anon Student Id', 'question_id', 'First Transaction Time']].values.tolist()
students = df['Anon Student Id'].unique().tolist()
len(students)

CPU times: user 906 ms, sys: 93.8 ms, total: 1 s
Wall time: 1 s


574

Transform time to datetime object

In [15]:
%%time
def create_time_object(x):
    x[3] = datetime.strptime(x[3], "%Y-%m-%d %H:%M:%S.0")
    
_ = list(map(create_time_object, data))

CPU times: user 6.44 s, sys: 125 ms, total: 6.56 s
Wall time: 6.53 s


In [16]:
with open('%s/kdd_2005_2006_train_data_time.pkl' % folder, 'wb') as pklfile:
    pickle.dump(data, pklfile)
with open('%s/kdd_2005_2006_students_time.pkl' % folder, 'wb') as pklfile:
    pickle.dump(students, pklfile)